![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# AI Service Deployment Notebook
This notebook contains steps and code to test, promote, and deploy an Agent as an AI Service.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>


Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Contents
This notebook contains the following parts:

1. Setup
2. Initialize all the variables needed by the AI Service
3. Define the AI service function
4. Deploy an AI Service
5. Test the deployed AI Service

## 1. Set up the environment

Before you can run this notebook, you must perform the following setup tasks:

### Connection to WML
This cell defines the credentials required to work with watsonx API for both the execution in the project, 
as well as the deployment and runtime execution of the function.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [15]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://eu-gb.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your api key (hit enter): ")
)



Please enter your api key (hit enter):  ········


In [16]:
client = APIClient(credentials)

### Connecting to a space
A space will be be used to host the promoted AI Service.


In [17]:
space_id = "07e9c7d9-bb6e-49e4-8503-94d090a24a86"
client.set.default_space(space_id)


'SUCCESS'

### Promote asset(s) to space
We will now promote assets we will need to stage in the space so that we can access their data from the AI service.


In [18]:
source_project_id = "1f760c18-c2a4-429d-9a92-cdc7d273939e"


## 2. Create the AI service function
We first need to define the AI service function

### 2.1 Define the function

In [19]:
params = {
    "space_id": "<YOUR_SPACE_ID>"
}

def gen_ai_service(context, params=params, **custom):
    from langchain_ibm import ChatWatsonx
    from ibm_watsonx_ai import APIClient
    from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
    from langchain_core.messages import AIMessage, HumanMessage
    from langgraph.checkpoint.memory import MemorySaver
    from langgraph.prebuilt import create_react_agent

    model_id = "meta-llama/llama-3-3-70b-instruct"
    service_url = "https://eu-gb.ml.cloud.ibm.com"

    # Credentials
    credentials = {
        "url": service_url,
        "token": context.generate_token()
    }
    client = APIClient(credentials)

    space_id = params.get("space_id")
    if not space_id:
        raise ValueError("space_id is required")
    client.set.default_space(space_id)

    # Create chat model
    def create_chat_model(watsonx_client):
        parameters = {
            "frequency_penalty": 0,
            "max_tokens": 1000,
            "presence_penalty": 0,
            "temperature": 0,
            "top_p": 1
        }
        return ChatWatsonx(
            model_id=model_id,
            url=service_url,
            space_id=space_id,
            params=parameters,
            watsonx_client=watsonx_client,
        )

    # Utility tool wrapper
    def create_utility_agent_tool(tool_name, config, api_client, **kwargs):
        from langchain_core.tools import StructuredTool
        utility_agent_tool = Toolkit(api_client=api_client).get_tool(tool_name)
        tool_description = kwargs.get("tool_description") or utility_agent_tool.get("agent_description") or utility_agent_tool.get("description")

        tool_schema = utility_agent_tool.get("input_schema") or {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }

        def run_tool(**tool_input):
            query = tool_input if utility_agent_tool.get("input_schema") else tool_input.get("input")
            results = utility_agent_tool.run(input=query, config=config)
            return results.get("output")

        return StructuredTool(
            name=tool_name,
            description=tool_description,
            func=run_tool,
            args_schema=tool_schema
        )

    # Create tools
    def create_tools(inner_client):
        tools = []
        tools.append(create_utility_agent_tool("GoogleSearch", None, inner_client))
        tools.append(create_utility_agent_tool("Wikipedia", {"maxResults": 5}, inner_client))
        tools.append(create_utility_agent_tool("DuckDuckGo", {}, inner_client))
        return tools

    # Create agent
    def create_agent(model, tools, messages):
        memory = MemorySaver()
        instructions = """You are a helpful AI career counseling assistant.
Students often struggle to make informed career decisions...
The challenge is to develop an intelligent, autonomous agent that monitors student performance, interests, and labor trends to deliver tailored career suggestions.
"""
        for message in messages:
            if message["role"] == "system":
                instructions += "\n" + message["content"]

        return create_react_agent(model, tools=tools, checkpointer=memory, state_modifier=instructions)

    # Convert chat messages
    def convert_messages(messages):
        converted = []
        for message in messages:
            if message["role"] == "user":
                converted.append(HumanMessage(content=message["content"]))
            elif message["role"] == "assistant":
                converted.append(AIMessage(content=message["content"]))
        return converted

    # Generate function
    def generate(context):
        payload = context.get_json()
        messages = payload.get("messages")
        inner_client = APIClient({"url": service_url, "token": context.get_token()})
        model = create_chat_model(inner_client)
        tools = create_tools(inner_client)
        agent = create_agent(model, tools, messages)

        generated_response = agent.invoke(
            {"messages": convert_messages(messages)},
            {"configurable": {"thread_id": "42"}}
        )

        last_message = generated_response["messages"][-1].content
        return {
            "headers": {"Content-Type": "application/json"},
            "body": {"choices": [{"index": 0, "message": {"role": "assistant", "content": last_message}}]}
        }

    # Streaming function
    def generate_stream(context):
        payload = context.get_json()
        messages = payload.get("messages")
        inner_client = APIClient({"url": service_url, "token": context.get_token()})
        model = create_chat_model(inner_client)
        tools = create_tools(inner_client)
        agent = create_agent(model, tools, messages)

        for chunk in agent.stream(
            {"messages": convert_messages(messages)},
            {"configurable": {"thread_id": "42"}},
            stream_mode=["updates", "messages"]
        ):
            yield chunk

    return generate, generate_stream


### 2.2 Test locally

In [20]:
# Initialize AI Service function locally
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)

streaming = False
findex = 1 if streaming else 0

# Pass space_id in params dict
local_function = gen_ai_service(context, params={"space_id": space_id})[findex]

# Example empty messages list
messages = []


In [21]:
local_question = "Change this question to test your function"

messages.append({ "role" : "user", "content": local_question })

context = RuntimeContext(api_client=client, request_payload_json={"messages": messages})

response = local_function(context)

result = ''

if (streaming):
    for chunk in response:
        print(chunk, end="\n\n", flush=True)
else:
    print(response)


Failure during Get available foundation models. (GET https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-07-09&space_id=07e9c7d9-bb6e-49e4-8503-94d090a24a86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"space_id 07e9c7d9-bb6e-49e4-8503-94d090a24a86 is not associated with a WML instance","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai#list-foundation-model-specs"}],"trace":"38fb913e034eadea849bf9f28b57bc10","status_code":403}
Unable to get model specifications from url: https://eu-gb.ml.cloud.ibm.com
Reason: Failure during Get available foundation models. (GET https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-07-09&space_id=07e9c7d9-bb6e-49e4-8503-94d090a24a86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","me

WMLClientError: Unable to get model specifications from url: https://eu-gb.ml.cloud.ibm.com
Reason: Failure during Get available foundation models. (GET https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-07-09&space_id=07e9c7d9-bb6e-49e4-8503-94d090a24a86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"space_id 07e9c7d9-bb6e-49e4-8503-94d090a24a86 is not associated with a WML instance","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai#list-foundation-model-specs"}],"trace":"38fb913e034eadea849bf9f28b57bc10","status_code":403}

## 3. Store and deploy the AI Service
Before you can deploy the AI Service, you must store the AI service in your watsonx.ai repository.

In [ ]:
# Look up software specification for the AI service
software_spec_id_in_project = "45f12dfe-aa78-5b8d-9f38-0ee223c47309"
software_spec_id = ""

try:
    software_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
except:
    software_spec_id = client.spaces.promote(software_spec_id_in_project, source_project_id, space_id)

In [ ]:
# Define the request and response schemas for the AI service
request_schema = {
    "application/json": {
        "$schema": "http://json-schema.org/draft-07/schema#",
        "type": "object",
        "properties": {
            "messages": {
                "title": "The messages for this chat session.",
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "role": {
                            "title": "The role of the message author.",
                            "type": "string",
                            "enum": ["user","assistant"]
                        },
                        "content": {
                            "title": "The contents of the message.",
                            "type": "string"
                        }
                    },
                    "required": ["role","content"]
                }
            }
        },
        "required": ["messages"]
    }
}

response_schema = {
    "application/json": {
        "oneOf": [{"$schema":"http://json-schema.org/draft-07/schema#","type":"object","description":"AI Service response for /ai_service_stream","properties":{"choices":{"description":"A list of chat completion choices.","type":"array","items":{"type":"object","properties":{"index":{"type":"integer","title":"The index of this result."},"delta":{"description":"A message result.","type":"object","properties":{"content":{"description":"The contents of the message.","type":"string"},"role":{"description":"The role of the author of this message.","type":"string"}},"required":["role"]}}}}},"required":["choices"]},{"$schema":"http://json-schema.org/draft-07/schema#","type":"object","description":"AI Service response for /ai_service","properties":{"choices":{"description":"A list of chat completion choices","type":"array","items":{"type":"object","properties":{"index":{"type":"integer","description":"The index of this result."},"message":{"description":"A message result.","type":"object","properties":{"role":{"description":"The role of the author of this message.","type":"string"},"content":{"title":"Message content.","type":"string"}},"required":["role"]}}}}},"required":["choices"]}]
    }
}

In [ ]:
# Store the AI service in the repository
ai_service_metadata = {
    client.repository.AIServiceMetaNames.NAME: "Career Counseling ",
    client.repository.AIServiceMetaNames.DESCRIPTION: "",
    client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: software_spec_id,
    client.repository.AIServiceMetaNames.CUSTOM: {},
    client.repository.AIServiceMetaNames.REQUEST_DOCUMENTATION: request_schema,
    client.repository.AIServiceMetaNames.RESPONSE_DOCUMENTATION: response_schema,
    client.repository.AIServiceMetaNames.TAGS: ["wx-agent"]
}

ai_service_details = client.repository.store_ai_service(meta_props=ai_service_metadata, ai_service=gen_ai_service)

In [ ]:
# Get the AI Service ID

ai_service_id = client.repository.get_ai_service_id(ai_service_details)

In [ ]:
# Deploy the stored AI Service
deployment_custom = {
    "avatar_icon": "ChatBot",
    "avatar_color": "backgroundBrand",
    "placeholder_image": "placeholder5.png"
}
deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Career Counseling ",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.CUSTOM: deployment_custom,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "You can ask any question regarding your career counseling.",
    client.repository.AIServiceMetaNames.TAGS: ["wx-agent"]
}

function_deployment_details = client.deployments.create(ai_service_id, meta_props=deployment_metadata, space_id=space_id)


## 4. Test AI Service

In [ ]:
# Get the ID of the AI Service deployment just created

deployment_id = client.deployments.get_id(function_deployment_details)
print(deployment_id)

In [ ]:
messages = []
remote_question = "Change this question to test your function"
messages.append({ "role" : "user", "content": remote_question })
payload = { "messages": messages }

In [ ]:
result = client.deployments.run_ai_service(deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

# Next steps
You successfully deployed and tested the AI Service! You can now view
your deployment and test it as a REST API endpoint.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  